In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = "Computer Vision Project/kits23"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1tnt3cKRdWn9lFKrM82KlrZH6CO5EgchD/Computer Vision Project/kits23


In [1]:
!pip install -q "monai-weekly[nibabel, tqdm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 71.5 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import albumentations as A

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary

from monai.networks.nets import SegResNet
from monai.losses import DiceLoss

In [ ]:
def augment(image, mask):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.ElasticTransform(alpha=1, sigma=50, p=0.5),
    ])
    augmented = transform(image=image, mask=mask)
    return augmented["image"], augmented["mask"]

In [ ]:
class CTDataset(Dataset):
    def __init__(self, dataframe, train, transform=None):
        self.data = dataframe
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx]['image']
        label_path = self.data.iloc[idx]['label']

        image = torch.load(image_path)
        label = torch.load(label_path)

        if self.train:
           image = image.squeeze(0)
           image, label = augment(image.numpy(), label.numpy())
           image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)
           label = torch.tensor(label, dtype=torch.long)

        # label = F.one_hot(label, num_classes=4).permute(3, 0, 1, 2).float()

        if self.transform:
           image, label = self.transform(image, label)

        return image, label

In [3]:
model = SegResNet(
    blocks_down=[1, 2, 2, 4, 4],
    blocks_up=[1, 1, 1, 1],
    init_filters=16,
    in_channels=1,
    out_channels=4,
    dropout_prob=0.2,
)

In [4]:
model.to('cpu')
input = (1, 128, 128, 128)
summary(model, input)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1    [-1, 16, 128, 128, 128]             432
         Dropout3d-2    [-1, 16, 128, 128, 128]               0
          Identity-3    [-1, 16, 128, 128, 128]               0
         GroupNorm-4    [-1, 16, 128, 128, 128]              32
              ReLU-5    [-1, 16, 128, 128, 128]               0
            Conv3d-6    [-1, 16, 128, 128, 128]           6,912
         GroupNorm-7    [-1, 16, 128, 128, 128]              32
              ReLU-8    [-1, 16, 128, 128, 128]               0
            Conv3d-9    [-1, 16, 128, 128, 128]           6,912
         ResBlock-10    [-1, 16, 128, 128, 128]               0
           Conv3d-11       [-1, 32, 64, 64, 64]          13,824
        GroupNorm-12       [-1, 32, 64, 64, 64]              64
             ReLU-13       [-1, 32, 64, 64, 64]               0
           Conv3d-14       [-1, 32, 64,

In [ ]:
class DC_and_CE_loss(nn.Module):
    def __init__(self, weight_ce=1.0, weight_dice=1.0):
        super(DC_and_CE_loss, self).__init__()
        self.weight_ce = weight_ce
        self.weight_dice = weight_dice
        self.dice_loss = DiceLoss()
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, net_output: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        target_one_hot = F.one_hot(target, 4).permute(0, 4, 1, 2, 3).float()
        dice_loss = self.dice_loss(net_output, target_one_hot)
        ce_loss = self.ce_loss(net_output, target)
        total_loss = self.weight_ce * ce_loss + self.weight_dice * dice_loss
        return total_loss

In [ ]:
train_image = [os.path.join("../preprocessed_image_train_2nd", path) for path in os.listdir("../preprocessed_image_train_2nd")]
train_segment = [os.path.join("../preprocessed_segment_train_2nd", path) for path in os.listdir("../preprocessed_segment_train_2nd")]

print(train_image)
print(train_segment)

print(len(train_image))
print(len(train_segment))

['../preprocessed_image_train_2nd/image_case_00236_0.pth', '../preprocessed_image_train_2nd/image_case_00236_1.pth', '../preprocessed_image_train_2nd/image_case_00236_2.pth', '../preprocessed_image_train_2nd/image_case_00236_3.pth', '../preprocessed_image_train_2nd/image_case_00236_4.pth', '../preprocessed_image_train_2nd/image_case_00236_5.pth', '../preprocessed_image_train_2nd/image_case_00236_6.pth', '../preprocessed_image_train_2nd/image_case_00236_7.pth', '../preprocessed_image_train_2nd/image_case_00236_8.pth', '../preprocessed_image_train_2nd/image_case_00236_9.pth', '../preprocessed_image_train_2nd/image_case_00051_0.pth', '../preprocessed_image_train_2nd/image_case_00051_1.pth', '../preprocessed_image_train_2nd/image_case_00051_2.pth', '../preprocessed_image_train_2nd/image_case_00051_3.pth', '../preprocessed_image_train_2nd/image_case_00051_4.pth', '../preprocessed_image_train_2nd/image_case_00051_5.pth', '../preprocessed_image_train_2nd/image_case_00051_6.pth', '../preproces

In [ ]:
train_df = pd.DataFrame({'image': train_image, 'label': train_segment})

dataset = CTDataset(train_df, train=True)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

criterion = DC_and_CE_loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print('loaded model')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load checkpoint if available
checkpoint_path = '../SegResNet3D_128_2nd/checkpoint_SegResNet_epoch_4.pth'
start_epoch = 1
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    start_epoch = checkpoint['epoch'] + 1
    print(f"Checkpoint loaded. Resuming training from epoch {start_epoch}.")

num_epochs = 5

if not os.path.exists('../SegResNet3D_128_2nd'):
   os.makedirs('../SegResNet3D_128_2nd')

# Training loop
for epoch in tqdm(range(start_epoch, num_epochs + 1)):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(dataloader, leave=True):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        print('loss', loss.item())

        running_loss += loss.item()

    print(f'Epoch [{epoch}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f"../SegResNet3D_128_2nd/checkpoint_SegResNet_epoch_{epoch}.pth")
    print(f'Checkpoint saved at epoch {epoch}.')

loaded model


<ipython-input-8-edf5aa96110b>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Checkpoint loaded. Resuming training from epoch 5.


  0%|          | 0/1560 [00:00<?, ?it/s]<ipython-input-4-2343cbc7ab94>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image = torch.load(image_path)
<ipython-input-4-2343

loss 0.0056100571528077126


...

loss 0.0902177020907402



100%|█████████▉| 1558/1560 [1:56:53<00:09,  4.62s/it]

loss 0.19873978197574615



100%|█████████▉| 1559/1560 [1:56:57<00:04,  4.43s/it]

loss 0.07361412793397903
